In [39]:
import tkinter as tk
import mysql.connector
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import random
from tkinter import messagebox
from datetime import datetime, timedelta

from PIL import Image, ImageTk


# Обновление балансов

In [40]:
def update_balances(today_date, cursor, conn):
    cursor.execute('SELECT * FROM transactions WHERE date_time > %s', (today_date,))
    transactions = cursor.fetchall()
        
    for transaction in transactions:
            amount = transaction[1]
            sender_account = transaction[3]
            recipient_account = transaction[4]
            
            # Обновление баланса отправителя
            cursor.execute('SELECT * FROM bank_accounts WHERE id = %s', (sender_account,))
            sender_account_info = cursor.fetchone()
            sender_balance = sender_account_info[2]
            
            # Если транзакция прошла через карту
            if transaction[5] is not None:
                cursor.execute('SELECT * FROM cards WHERE id = %s' % transaction[5])
                card_info = cursor.fetchone()
                if card_info[5] != None and (card_info[5] - amount < 0 or card_info[6] - amount < 0):
                    continue
                
                cursor.execute('UPDATE bank_accounts SET balance = %s WHERE id = %s' % (sender_balance - amount, sender_account))
                
            # Обновление баланса получателя
            cursor.execute('SELECT * FROM bank_accounts WHERE id = %s' % recipient_account)
            recipient_account_info = cursor.fetchone()
            recipient_balance = recipient_account_info[2]
            cursor.execute('UPDATE bank_accounts SET balance = %s WHERE id = %s' % (recipient_balance + amount, recipient_account))
            
            # Подтверждение изменений
            conn.commit()
    
    cursor.execute('SELECT * FROM cards WHERE daily_limit is not NULL')
    cards = cursor.fetchall()

    cursor.execute('SELECT MAX(date_time) FROM transactions')
    last_month = cursor.fetchone()[0].month

    # Обновление остатков дневных и месячных лимитов
    for card in cards:
        cursor.execute('UPDATE cards SET daily_limit_left = %s WHERE id = %s', (card[3], card[0]))

        if last_month < today_date.month:
            cursor.execute('UPDATE cards SET monthly_limit_left = %s WHERE id = %s', (card[4], card[0]))
    

# Запросы

In [41]:
def find_account_with_max_balance(cursor):
    cursor.callproc("find_account_with_max_balance")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0][0]


In [42]:
def find_double_purchase(cursor):
    cursor.callproc("find_double_purchase")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [43]:
def find_account_with_max_cards(cursor):
    cursor.callproc("find_account_with_max_cards")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0][0]

In [44]:
def find_inactive_atm(cursor):
    cursor.callproc("find_inactive_atm")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [45]:
def find_overlimit(cursor):
    cursor.callproc("find_overlimit")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [46]:
def find_transactions_in_non_working_hours(cursor):
    cursor.callproc("find_transactions_in_non_working_hours")
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [47]:
def find_card_with_most_transactions(cursor, search_date):
    cursor.callproc("find_card_with_most_transactions", (search_date,))
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [48]:
def find_shop_with_max_orders(cursor, start_date, end_date):
    cursor.callproc("find_shop_with_max_orders", (start_date, end_date))
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [49]:
def transaction_statistics(cursor, num_days, commercial_enitity_id, search_date):
    cursor.callproc("transaction_statistics", (num_days, commercial_enitity_id, search_date))
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

In [50]:
def citizen_statistics(cursor, citizen_id, start_date, end_date):
    cursor.callproc("citizen_statistics", (citizen_id, start_date, end_date))
    results = []
    for result in cursor.stored_results():
        results.append(result.fetchall())
    return results[0]

# Заполнение транзакциями

In [51]:
def fill_transactions(start_date, cursor, conn):

    # Генерируем случайно количество транзакций
    n_transactions = random.randint(1, 1000)

    # Получаем список потенциальных получателей
    cursor.execute("SELECT id FROM bank_accounts WHERE account_type IN (%s, %s)", (1, 3))
    receivers_accounts = [account[0] for account in cursor.fetchall()]

    # Получаем список потенциальных пользователей карт 
    cursor.execute("SELECT id FROM bank_accounts WHERE account_type = 2")
    sender_accounts = [account[0] for account in cursor.fetchall()]

    for _ in range(n_transactions): 
        try:
            # Генерируем сумму
            amount = random.randint(1, 500) * 5

            # Генерируем случайные дату и время с учетом условия
            if random.random() < 0.9:  # 90% шанс
                # Генерируем случайное время до 22:00
                hour = random.randint(0, 21)
            else:
                # Генерируем случайное время после 22:00
                hour = random.randint(22, 23)

            # Генерируем случайное время в пределах выбранного часа
            datetime = start_date + timedelta(hours=hour, minutes=random.randint(0, 59), seconds=random.randint(0, 59))

            # Форматируем результат в строку
            datetime = datetime.strftime("%Y-%m-%d %H:%M:%S")

            # Выбираем случайный счет отправителя
            probs = [0.25, 0.75]

            # Если операция пополнения наличными
            if random.choices([1, 2], weights=probs)[0] == 1:
                amount = (amount + 50) // 100 * 100

                sender_account = 1
                recipient_account = random.choice(sender_accounts)

                # Выбираем случайную карту привязнную к счету
                cursor.execute("SELECT * FROM cards WHERE account_id = %s", (recipient_account,))
                cards = [card[0] for card in cursor.fetchall()]
                card_id = random.choice(cards)

                # Выбираем случайный банкомат
                cursor.execute("SELECT * FROM atms")
                atms = [atm[0] for atm in cursor.fetchall()]
                atm_id = random.choice(atms)
                
                terminal_id = None
            
            # Если операция снятия наличных или любая другая
            else:
                sender_account = random.choice(sender_accounts)

                # Выбираем случайную карту привязнную к счету
                cursor.execute("SELECT * FROM cards WHERE account_id = %s", (sender_account,))
                cards = [card[0] for card in cursor.fetchall()]
                card_id = random.choice(cards)

                # Выбираем случайный счет получателя
                probs = [0.10, 0.90]
                if random.choices([1, 2], weights=probs)[0] == 1 and sender_account!= 1:
                # Если снятие наличных
                    recipient_account = 1
                    amount = (amount + 50) // 100 * 100
                else:
                    recipient_account = random.choice(receivers_accounts)

                # Достаем тип счета 
                cursor.execute("SELECT account_type FROM bank_accounts WHERE id = %s", (recipient_account,))
                account_type = [account_type[0] for account_type in cursor.fetchall()][0]
                
                # Если операция по терминалу
                if account_type == 1:
                    # Выбираем случайный терминал привязанный к счету
                    cursor.execute("SELECT * FROM terminals WHERE account_id = %s", (recipient_account,))
                    terminals = [terminal[0] for terminal in cursor.fetchall()]
                    terminal_id = random.choice(terminals)

                    atm_id = None 

                # Если операция снятия наличных
                elif account_type == 4:
                    # Выбираем случайный банкомат
                    cursor.execute("SELECT * FROM atms")
                    atms = [atm[0] for atm in cursor.fetchall()]
                    atm_id = random.choice(atms)

                    terminal_id = None

                # Если операция в интернет магазине
                elif account_type == 3:

                    atm_id = None 
                    terminal_id = None

            
            print(f"Amount: {amount}, DateTime: {datetime}, Sender_account: {sender_account}, Card: {card_id}, Terminal: {terminal_id}, Receiver account: {recipient_account}, ATM_id: {atm_id} ")    
            cursor.execute('''
                INSERT INTO transactions (amount, date_time, sender_account, recipient_account, card_id, atm_id, terminal_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            ''', (amount, datetime, sender_account, recipient_account, card_id, atm_id, terminal_id))
            print((amount, datetime, sender_account, recipient_account, card_id, atm_id, terminal_id))
            
            # Подтверждаем изменения в базе данных
            conn.commit()

        except Exception as e:
            print(e)
            continue

# Переход на следующий день

In [52]:
def skip_day(app, conn, cursor, start_date):
    start_date = datetime.strptime(start_date, '%d.%m.%Y') + timedelta(days=1)

    fill_transactions(start_date, cursor, conn)
    update_balances(start_date, cursor, conn)
    create_admin_interface(app, cursor, conn)

# Добавление даты

In [53]:
def add_date(cursor, app):
    # Создание Label со сегодняшней датой
    cursor.execute("SELECT MAX(date_time) FROM transactions")
    last_transaction_date = cursor.fetchone()[0]

    # Форматирование даты в нужный формат
    last_transaction_date = last_transaction_date.strftime('%d.%m.%Y')
    date_label = tk.Label(app, text="Сегодняшняя дата - {}".format(last_transaction_date), bg='black', fg='white', font=("futura", 14, "bold"))
    date_label.pack(anchor='n', pady=10)
    date_label.config(highlightbackground="red", highlightthickness=2)

    return last_transaction_date

# Конвертация даты

In [54]:
def convert_to_yyyy_mm_dd(date_str):
    try:
        print(date_str)
        if date_str[4] == '-' and date_str[7]== '-':
            return date_str
        # Преобразование строки в объект datetime
        date_obj = datetime.strptime(date_str, '%d.%m.%Y')  # Здесь '%d.%m.%Y' - формат входной даты
        # Форматирование даты в формат 'yyyy-mm-dd'
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        messagebox.showerror("Ошибка", "Некорректный формат даты!")
        return None

# Основной интерфейс

In [55]:
# Функция для очистки виджетов
def clear_widgets(app):
    for widget in app.winfo_children():
        widget.destroy()

# Функция для создания интерфейса администратора
def create_admin_interface(app, cursor, conn):
    clear_widgets(app)

    last_transaction_date = add_date(cursor, app)

    # Создание главного фрейма для интерфейса администратора
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Создание кнопки для запросов
    request_button = tk.Button(main_frame, text="Запросы", bg='red', fg='white',
                             font=("futura", 14, "bold"), command=lambda: create_requests_interface(app, cursor, conn))
    request_button.pack(pady=20)

    # Создание кнопки для графиков
    graphs_button = tk.Button(main_frame, text="Графики", bg='red', fg='white',
                            font=("futura", 14, "bold"), command=lambda: create_graphs_interface(app, cursor, conn))
    graphs_button.pack(pady=20)

    # Кнопка для перехода на день вперед
    forward_button = tk.Button(app, text="Перейти на день вперед", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: skip_day(app, conn, cursor, last_transaction_date))
    forward_button.place(x=0, y=0)

def create_graphs_interface(app, cursor, conn):
    clear_widgets(app)

    # Создание главного фрейма для интерфейса графиков
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Создание кнопок графиков
    request_button = tk.Button(main_frame, text="График суммы всех транзакций по дням", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_graph_1_screen(app, cursor, conn))
    request_button.grid(row=0, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Гистограмма суммы продаж по предприятиям", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_graph_2_screen(app, cursor, conn))
    request_button.grid(row=1, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Зависимость количества транзакций от времени", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_graph_3_screen(app, cursor, conn))
    request_button.grid(row=2, column=0, padx=10, pady=10)
    
    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=lambda: create_admin_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_graph_1_screen(app, cursor, conn):
    clear_widgets(app)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="График суммы транзакций по дням", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    # Получаем данные о транзакциях из базы данных
    cursor.execute("SELECT DATE(date_time), SUM(amount) FROM transactions GROUP BY DATE(date_time)")
    data = cursor.fetchall()

    # Разбиваем данные на даты и суммы транзакций
    dates = [row[0].strftime("%d.%m.%Y") for row in data]
    amounts = [row[1] for row in data]

    # Создаем график
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.plot(dates, amounts)
    ax.set_xlabel('Дата')
    ax.set_ylabel('Сумма транзакций')
    ax.grid(True)

    # Вставляем график в окно tkinter
    canvas = FigureCanvasTkAgg(fig, master=app)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, expand=1)


    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_graphs_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_graph_2_screen(app, cursor, conn):
    clear_widgets(app)
    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Гистограмма суммы продаж по предприятиям", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)
    
 # Получаем данные для графика
    cursor.execute('''
        SELECT ce.name, SUM(t.amount) AS total_sales
        FROM commercial_entities ce
        LEFT JOIN bank_accounts ba ON ce.id = ba.commercial_entity_id
        LEFT JOIN transactions t ON ba.id = t.recipient_account
        GROUP BY ce.id
    ''')
    rows = cursor.fetchall()
    
    # Извлечение данных из результатов запроса
    companies = [row[0] for row in rows]
    sales = [row[1] for row in rows]

# Создаем график
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.bar(companies, sales, align='center', alpha=0.5)
    ax.set_xlabel('Предприятие')
    ax.set_ylabel('Сумма продаж')
    ax.grid(True)

    # Вставляем график в окно tkinter
    canvas = FigureCanvasTkAgg(fig, master=app)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, expand=1)

        # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_graphs_interface(app, cursor, conn))
    back_button.place(x=0, y=0)


def create_graph_3_screen(app, cursor, conn):
    clear_widgets(app)
    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Точечная диаграмма зависимости количества транзакций от времени суток", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)
    
 # Получаем данные для графика
    cursor.execute('''
        SELECT HOUR(date_time) AS hour, COUNT(*) AS transactions_count
        FROM transactions
        GROUP BY HOUR(date_time)
    ''')
    rows = cursor.fetchall()
    
    # Извлечение данных из результатов запроса
    hours = [row[0] for row in rows]
    transactions_count = [row[1] for row in rows]

    # Создание точечной диаграммы
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.scatter(hours, transactions_count, color='blue')
    ax.set_xlabel('Время (часы)')
    ax.set_ylabel('Количество транзакций')
    ax.grid(True)

    # Вставляем график в окно tkinter
    canvas = FigureCanvasTkAgg(fig, master=app)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, expand=1)

        # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_graphs_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

    
def create_requests_interface(app, cursor, conn):
    clear_widgets(app)

    # Создание главного фрейма для интерфейса запросов
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Создание кнопок запросов
    request_button = tk.Button(main_frame, text="Вывести счет с максимальным балансом", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_1_screen(app, cursor, conn))
    request_button.grid(row=0, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Вывести счет с максимальным количеством карт", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_2_screen(app, cursor, conn))
    request_button.grid(row=1, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text='''Поиск "Двойных покупок"''', bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_3_screen(app, cursor, conn))
    request_button.grid(row=2, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Поиск поломанного банкомата", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_4_screen(app, cursor, conn))
    request_button.grid(row=3, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Поиск операций которые вышли за лимиты карты", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_5_screen(app, cursor, conn))
    request_button.grid(row=4, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Поиск карты с максимальным количеством операций", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_6_screen(app, cursor, conn))
    request_button.grid(row=5, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Поиск операций проведенных в нерабочее время предприятий".format(7), bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_7_screen(app, cursor, conn))
    request_button.grid(row=6, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Поиск интернет магазинов с наибольшим количеством продаж".format(8), bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_8_screen(app, cursor, conn))
    request_button.grid(row=7, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Статистика по продажам предприятий".format(9), bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_9_screen(app, cursor, conn))
    request_button.grid(row=8, column=0, padx=10, pady=10)

    request_button = tk.Button(main_frame, text="Статистика покупок гражданина", bg='red', fg='white',
                                font=("futura", 14, "bold"), command=lambda: create_request_10_screen(app, cursor, conn))
    request_button.grid(row=9, column=0, padx=10, pady=10)


    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=lambda: create_admin_interface(app, cursor, conn))
    back_button.place(x=0, y=0)


def create_request_1_screen(app, cursor, conn):
    clear_widgets(app)

    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        result = find_account_with_max_balance(cursor)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        # Вставка данных в таблицу
        table.insert("", "end", values=result)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на максимальный баланс счета", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Номер счета', 'Баланс'), show='headings')
    table.heading('ID', text='ID')
    table.heading('Номер счета', text='Номер счета')
    table.heading('Баланс', text='Баланс')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_2_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        result = find_account_with_max_cards(cursor)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        # Вставка данных в таблицу
        table.insert("", "end", values=result)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на максимальное количество карточек", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Номер счета', 'Количество карточек'), show='headings')
    table.heading('ID', text='ID')
    table.heading('Номер счета', text='Номер счета')
    table.heading('Количество карточек', text='Количество карточек')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)


def create_request_3_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        try:
            result = find_double_purchase(cursor)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск двойной покупки", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID_1', 'Сумма первой транзакции', 'Дата и время первой транзакции', 'ID_2', 'Сумма второй транзакции', 'Дата и время второй транзакции'), show='headings')
    table.heading('ID_1', text='ID_1')
    table.heading('Сумма первой транзакции', text='Сумма первой транзакции')
    table.heading('Дата и время первой транзакции', text='Дата и время первой транзакции')
    table.heading('ID_2', text='ID_2')
    table.heading('Сумма второй транзакции', text='Сумма второй транзакции')
    table.heading('Дата и время второй транзакции', text='Дата и время второй транзакции')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_4_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        try:
            result = find_inactive_atm(cursor)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск поломанного банкомата", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Координаты'), show='headings')
    table.heading('ID', text='ID')
    table.heading('Координаты', text='Координаты')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_5_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        try:
            result = find_overlimit(cursor)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск операций которые выходят за лимиты по карте", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID транзакции', 'Сумма транзакции', 'Дата и время', 'ID карты', 'Дневной лимит', 'Месячный лимит', 'Дневной лимит (осталось)', 'Месячный лимит (осталось)'), show='headings')
    table.heading('ID транзакции', text='ID транзакции')
    table.heading('Сумма транзакции', text='Сумма транзакции')
    table.heading('Дата и время', text='Дата и время')
    table.heading('ID карты', text='ID карты')
    table.heading('Дневной лимит', text='Дневной лимит')
    table.heading('Месячный лимит', text='Месячный лимит')
    table.heading('Дневной лимит (осталось)', text='Дневной лимит (осталось)')
    table.heading('Месячный лимит (осталось)', text='Месячный лимит (осталось)')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_6_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        date_value = date_entry.get()
        date_value = convert_to_yyyy_mm_dd(date_value)
        
        try:
            result = find_card_with_most_transactions(cursor, date_value)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск поиск карты с максимальным количеством операций", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Количество операций'), show='headings')
    table.heading('ID', text='ID')
    table.heading('Количество операций', text='Количество операций')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создаем поле для ввода даты
    date_entry = tk.Label(buttons_frame, width=15, text="Введите дату:", bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10, pady=20)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)


def create_request_7_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        try:
            result = find_transactions_in_non_working_hours(cursor)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск операций проведенных в нерабочее время предприятий", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Сумма операции', 'Дата и время', 'Время открытия', 'Время закрытия' ), show='headings')
    table.heading('ID', text='ID')
    table.heading('Сумма операции', text='Сумма операции')
    table.heading('Дата и время', text='Дата и время')
    table.heading('Время открытия', text='Время открытия')
    table.heading('Время закрытия', text='Время закрытия')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_8_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        date_value_1 = date_entry_1.get()
        date_value_1 = convert_to_yyyy_mm_dd(date_value_1)

        date_value_2 = date_entry_2.get()
        date_value_2 = convert_to_yyyy_mm_dd(date_value_2)
        try:
            result = find_shop_with_max_orders(cursor, date_value_1, date_value_2)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Запрос на поиск интернет-магазинов с самым большим объемом продаж", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID', 'Название', 'Количество заказов'), show='headings')
    table.heading('ID', text='ID')
    table.heading('Название', text='Название')
    table.heading('Количество заказов', text='Количество заказов')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создаем поле для ввода даты
    date_entry = tk.Label(buttons_frame, width=15, text="Начало периода:", bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry_1 = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry_1.pack(side='top', padx=10, pady=20)

        # Создаем поле для ввода даты
    date_entry = tk.Label(buttons_frame, width=15, text="Конец периода:", bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry_2 = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry_2.pack(side='top', padx=10, pady=20)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

def create_request_9_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        num_days = num_days_entry.get()
        commercial_enitity_id = commercial_enitity_id_entry.get()
        date_value = date_entry.get()
        date_value = convert_to_yyyy_mm_dd(date_value)
        try:
            result = transaction_statistics(cursor, num_days, commercial_enitity_id, date_value)
            print(result)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Статистика по продажам предприятия", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)

    last_transaction_date = add_date(cursor, app)

    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('Название предприятия', 'Средний чек', 'Максимальный чек', 'Количество покупок'), show='headings')
    table.heading('Название предприятия', text='Название предприятия')
    table.heading('Средний чек', text='Средний чек')
    table.heading('Максимальный чек', text='Максимальный чек')
    table.heading('Количество покупок', text='Количество покупок')
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)

    # Создаем поле для ввода даты
    num_days = tk.Label(buttons_frame, width=15, text="Количество дней", bg='red', fg='white', font=("futura", 12))
    num_days.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    num_days_entry = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    num_days_entry.pack(side='top', padx=10, pady=20)

        # Создаем поле для ввода даты
    commercial_enitity_id = tk.Label(buttons_frame, width=15, text="ID предприятия", bg='red', fg='white', font=("futura", 12))
    commercial_enitity_id.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    commercial_enitity_id_entry = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    commercial_enitity_id_entry.pack(side='top', padx=10, pady=20)

        # Создаем поле для ввода даты
    date_value = tk.Label(buttons_frame, width=15, text="Дата отсчета", bg='red', fg='white', font=("futura", 12))
    date_value.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10, pady=20)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)


def create_request_10_screen(app, cursor, conn):
    clear_widgets(app)
    
    # Функция для выполнения запроса и обновления таблицы
    def execute_query():
        # Выполнение запроса
        citizen_id = citizen_id_entry.get()
    
        date_value_1 = date_entry_1.get()
        date_value_1 = convert_to_yyyy_mm_dd(date_value_1)

        date_value_2 = date_entry_2.get()
        date_value_2 = convert_to_yyyy_mm_dd(date_value_2)
        
        try:
            result = citizen_statistics(cursor, citizen_id, date_value_1, date_value_2)
        except mysql.connector.Error as err:
            messagebox.showerror("Ошибка", err.msg)

        # Очистка данных в таблице перед обновлением
        for row in table.get_children():
            table.delete(row)

        print(result)
        for i in result:
            table.insert("", "end", values=i)
    
        # Функция для очистки таблицы
    def clear_table():
        # Очистка данных в таблице
        for row in table.get_children():
            table.delete(row)

    # Создание главного фрейма для экрана запроса 1
    main_frame = tk.Frame(app, bg='black')
    main_frame.place(anchor='c', relx=0.5, rely=0.5)

    # Закрепленный сверху по центру лейбл с красной границей
    label = tk.Label(app, text="Статистика по транзакциям гражданина", bg='black', fg='white', font=("futura", 14, "bold"))
    label.config(highlightbackground="red", highlightthickness=2)
    label.pack(anchor='n', pady=10)
    
    last_transaction_date = add_date(cursor, app)
    
    # Создание таблицы для отображения результатов запроса
    table = ttk.Treeview(main_frame, columns=('ID гражданина', 'Средняя сумма транзакции', 'Любимый интернет магазин', "Количество операций в любимом ИМ", "Любимое предприятие", "Количество операций в любимом предприятии"), show='headings')
    table.heading('ID гражданина', text='ID гражданина')
    table.heading('Средняя сумма транзакции', text='Средняя сумма транзакции')
    table.heading('Любимый интернет магазин', text='Любимый интернет магазин')
    table.heading("Количество операций в любимом ИМ", text="Количество операций в любимом ИМ")
    table.heading("Любимое предприятие", text="Любимое предприятие")
    table.heading("Количество операций в любимом предприятии", text="Количество операций в любимом предприятии")
    table.pack(pady=20)

    # Создание фрейма для кнопок
    buttons_frame = tk.Frame(app, bg='black')
    buttons_frame.pack(side='bottom', pady=10)
    
    citizen_id_entry = tk.Label(buttons_frame, width=15, text="ID гражданина:", bg='red', fg='white', font=("futura", 12))
    citizen_id_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    citizen_id_entry = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    citizen_id_entry.pack(side='top', padx=10, pady=20)

    # Создаем поле для ввода даты
    date_entry = tk.Label(buttons_frame, width=15, text="Начало периода:", bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry_1 = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry_1.pack(side='top', padx=10, pady=20)

        # Создаем поле для ввода даты
    date_entry = tk.Label(buttons_frame, width=15, text="Конец периода:", bg='red', fg='white', font=("futura", 12))
    date_entry.pack(side='top', padx=10)

    # Создаем поле для ввода даты
    date_entry_2 = tk.Entry(buttons_frame, width=15, bg='red', fg='white', font=("futura", 12))
    date_entry_2.pack(side='top', padx=10, pady=20)

    # Создание кнопки "Выполнить запрос"
    execute_button = tk.Button(buttons_frame, text="Выполнить запрос", bg='red', fg='white', font=("futura", 12, "bold"),
                               command=execute_query)
    execute_button.pack(side='left', padx=10)

    # Создание кнопки "Очистить таблицу"
    clear_button = tk.Button(buttons_frame, text="Очистить таблицу", bg='red', fg='white', font=("futura", 12, "bold"),
                             command=clear_table)
    clear_button.pack(side='left', padx=10)

    # Кнопка для перехода назад
    back_button = tk.Button(app, text="Назад", bg='red', fg='white', font=("futura", 12, "bold"),
                            command=lambda: create_requests_interface(app, cursor, conn))
    back_button.place(x=0, y=0)

In [56]:
def on_closing(app, conn):
    # Закрытие соединения с базой данных
    conn.close()
    app.destroy()

In [57]:
def main():

    # Подключение к базе данных
    conn = mysql.connector.connect(
        host="localhost",
        port = "3306",
        user="root",
        password="8&5jypSfzFyDhL",
        database = 'hobbiton_bank'
    )

    # Создание курсора для выполнения SQL-запросов
    cursor = conn.cursor(buffered=True)
    
    app = tk.Tk()
    # Установка фона окна черным
    app.configure(bg='black')
    # Загрузка изображения
    image = Image.open("image.jpg")
    bg_image = ImageTk.PhotoImage(image)

    # Создание холста
    canvas = tk.Canvas(app, width=image.width, height=image.height)
    canvas.pack(fill="both", expand=True)

    # Размещение изображения на холсте
    canvas.create_image(0, 0, image=bg_image, anchor="nw")
    app.protocol("WM_DELETE_WINDOW", lambda: on_closing(app, conn))
    # Установка размеров окна на 1000*1000
    app.geometry("1000x1000")
    app.title("Служба безопасности банка Хоббитон")
    
    create_admin_interface(app, cursor, conn)
    
    app.mainloop()

main()